In [1]:
import data_set
import model
import data_helper

In [2]:
bert_path = r"C:\Users\Administrator\Desktop\研究生相关\项目-问题文本分类\Bert-Pytorch-Chinese-TextClassification-master\bert\bert-tiny-chinese-6L"
label_dict = {"负向": {"label_words": ["不","不太"], "label_id": 0},
                  "正向": {"label_words": ["很", "非常","比较","挺"], "label_id": 1}}

MAX_LEN = 256
template = "{mask}满意。{text}"



In [3]:
from transformers import BertTokenizer

tk = BertTokenizer.from_pretrained(bert_path)

In [4]:
Bert = model.PromptModel.from_pretrained(bert_path)

Some weights of the model checkpoint at C:\Users\Administrator\Desktop\研究生相关\项目-问题文本分类\Bert-Pytorch-Chinese-TextClassification-master\bert\bert-tiny-chinese-6L were not used when initializing PromptModel: ['bert.pooler.dense.weight', 'fit_dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'fit_dense.weight']
- This IS expected if you are initializing PromptModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing PromptModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of PromptModel were not initialized from the model checkpoint at C:\Users\Administrator\Desktop\研究生相关\项目-问题文本分类\Bert-Pytorch-Chinese-TextClassification-master\bert\bert-tin

In [5]:
trainSet = data_set.PromptDataSet(tk,MAX_LEN,template,label_dict,data_dir=".\data",
                                  data_set_name="train",
                                  path_file=r".\data\train.json")
testSet = data_set.PromptDataSet(tk,MAX_LEN,template,label_dict,data_dir=".\data",
                                  data_set_name="test",
                                  path_file=r".\data\test.json")

In [6]:

sample = [trainSet[i] for i in range(32)]
batch = data_set.collate_func(sample)
res = Bert.forward(input_ids=batch["input_ids"], 
                   attention_mask=batch["attention_mask"], 
                   mask_index=batch["mask_index"],
                   token_handler="mean",
                   words_ids=trainSet.words_ids, 
                   words_ids_mask=trainSet.words_ids_mask,
                   label=batch["label"])

"""
res:{
    0:loss,
    1:logits,
    2:predict_label
}
"""

D:\git_projects\ChatGPTBook\PromptProj\model.py:55: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  mask = mask_index + torch.range(0, logits_shapes[0] - 1, dtype=torch.long, device=logits.device) * \


'\nres:{\n    0:loss,\n    1:logits,\n    2:predict_label\n}\n'

In [21]:
verb = trainSet.get_verbalizer()

params = list(Bert.named_parameters())

for i in params:
    print(f"layer:{i[0]:<30}  ->  params:{i[1].shape}")
    
    
    

layer:bert.embeddings.word_embeddings.weight  ->  params:torch.Size([21128, 768])
layer:bert.embeddings.position_embeddings.weight  ->  params:torch.Size([512, 768])
layer:bert.embeddings.token_type_embeddings.weight  ->  params:torch.Size([2, 768])
layer:bert.embeddings.LayerNorm.weight  ->  params:torch.Size([768])
layer:bert.embeddings.LayerNorm.bias  ->  params:torch.Size([768])
layer:bert.encoder.layer.0.attention.self.query.weight  ->  params:torch.Size([768, 768])
layer:bert.encoder.layer.0.attention.self.query.bias  ->  params:torch.Size([768])
layer:bert.encoder.layer.0.attention.self.key.weight  ->  params:torch.Size([768, 768])
layer:bert.encoder.layer.0.attention.self.key.bias  ->  params:torch.Size([768])
layer:bert.encoder.layer.0.attention.self.value.weight  ->  params:torch.Size([768, 768])
layer:bert.encoder.layer.0.attention.self.value.bias  ->  params:torch.Size([768])
layer:bert.encoder.layer.0.attention.output.dense.weight  ->  params:torch.Size([768, 768])
layer:b